In [1]:
import yfinance as yf
import pandas as pd
from pathlib import Path
import math 

In [2]:
result_folder = Path.cwd().parent / 'data'

symbols = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker'].tolist()
symbols = pd.Series(symbols).unique()
backup = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker']
if len(backup) > 0:
    backup.to_csv(result_folder / 'known_tickers.csv', index=False)
owned_tickers = pd.read_csv(result_folder / 'owned_tickers.csv')['Ticker'].tolist()

In [3]:
df = pd.DataFrame(columns=[
    'Ticker', 
    'short_name', 
    'in_portfolio',
    'value_score',  
    'recommendation_score',
    'analyst_mean%', 
    'insider_buy%', 
    'sector', 
    'industry', 
    'country',
    'P/E (TTM)', 
    'ROA%',
])

for symbol in symbols: 
    ticker = yf.Ticker(symbol)
    try:
        in_portfolio = 'No'
        if symbol in owned_tickers:
            in_portfolio = 'Yes'

        analyst_mean = 0
        if 'mean' in ticker.analyst_price_targets and ticker.info['currentPrice']:
            analyst_mean = ticker.analyst_price_targets['mean'] / ticker.info['currentPrice'] - 1
        
        roa = ticker.info['returnOnAssets']*100
        pe = ticker.info['trailingPE']
        
        value_score = format(2.5 + math.log(roa+5) - math.log(pe+25), '.2f')
        
        insider_buy = 0
        if 'Shares' in ticker.insider_purchases.columns:
            insider_buy = format(ticker.insider_purchases.loc[ticker.insider_purchases.index[4], 'Shares'], '.2f')
            if insider_buy == '<NA>':
                insider_buy = 0

        recommendation_score = format(float(value_score) + analyst_mean * 1.25 + float(insider_buy) * 2.0, '.2f')

        df.loc[symbol] = [
            symbol, 
            ticker.info['shortName'], 
            in_portfolio,
            value_score,
            recommendation_score,
            format(analyst_mean * 100, '.2f'), 
            insider_buy,
            ticker.info['sector'], 
            ticker.info['industry'], 
            ticker.info['country'],
            format(pe, '.2f') if isinstance(pe, (int, float)) else 'N/A', 
            format(roa, '.2f') if isinstance(roa, (int, float)) else 'N/A',
        ]
    except Exception as e:
        print(f"Error with {symbol}: {e}. Removed Ticker from list.")

df.to_csv(result_folder / 'simple_screener_results.csv', index=False)
display(df)

print(f'Got data for {len(df)} stocks')

,Ticker,short_name,in_portfolio,value_score,recommendation_score,analyst_mean%,insider_buy%,sector,industry,country,P/E (TTM),ROA%
MATAS.CO,MATAS.CO,Matas A/S,No,0.85,1.29,35.34,0,Consumer Cyclical,Specialty Retail,Denmark,21.11,3.84
TRIFOR.CO,TRIFOR.CO,Trifork Group AG,No,0.74,1.17,34.28,0,Technology,Information Technology Services,Switzerland,13.29,1.59
QQ.L,QQ.L,QINETIQ GROUP PLC ORD 1P,Yes,1.26,1.58,25.45,0,Industrials,Aerospace & Defense,United Kingdom,16.11,6.89
RNMBY,RNMBY,Rheinmetall AG,Yes,0.28,0.28,0.00,0,Industrials,Aerospace & Defense,Germany,83.33,6.81
SAABF,SAABF,Saab AB,Yes,0.22,0.22,0.00,0,Industrials,Aerospace & Defense,Sweden,59.97,3.70
BCKIY,BCKIY,Babcock International Group Plc,No,0.99,0.99,0.00,0,Industrials,Engineering & Construction,United Kingdom,20.48,5.08
BAESY,BAESY,BAE Systems PLC,No,0.78,0.57,-16.59,0,Industrials,Aerospace & Defense,United Kingdom,27.67,4.40
IVSO.ST,IVSO.ST,INVISIO AB,No,1.36,1.40,3.22,0,Technology,Communication Equipment,Sweden,54.32,20.31
NSKFF,NSKFF,Kongsberg Gruppen ASA,Yes,0.56,0.56,0.00,0,Industrials,Aerospace & Defense,Norway,55.89,6.65
GMAB,GMAB,Genmab A/S,Yes,1.69,2.27,46.58,0,Healthcare,Biotechnology,Denmark,10.89,10.91


Got data for 34 stocks
